### ML PYspark

The first thing to do is to create a .env file in the root of the directory. Add to the file the following two varibles 
ACCESS_KEY, ACCESS_SECRET. 
Check for more detailed explanation here: [dotenv]("https://pypi.org/project/python-dotenv/), he explains how the .env should look like. After that, the variables are add to the os.environ and can be access as a simple dict structure

In [31]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as spark_sum, when, input_file_name
from functools import reduce
import sys
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler
import os
from pyspark.ml.classification import RandomForestClassifier
from dotenv import load_dotenv
from sparkmeasure import StageMetrics

In [32]:
import random
## load .env
load_dotenv()


True

## Console Login

The following classes are to handle the spark on the AWS 

In [33]:
from src.s3handler import Sparker, PreProcessing, FeatureEngineering

In [34]:
## Initialize the class
spark = Sparker(os.environ['ACCESS_KEY'],os.environ['ACCESS_SECRET'])

## local session
session = spark._create_local_session()

## Read parquet

In [35]:
parquet_cols = ["xyz","Intensity","Classification","Red","Green","Blue","Infrared","ReturnNumber","NumberOfReturns"]

## Read the parquet and stored it 
df = spark.read_parquet("ubs-datasets",
                    "FRACTAL/data/train/TRAIN-0436_6399-002955400.parquet",
                    read_all=False) \
                    .select(*parquet_cols)

# # Read the list of parquet files
# list_s3 = ["FRACTAL/data/train/TRAIN-1200_6136-008972557.parquet", "FRACTAL/data/train/TRAIN-0436_6399-002955400.parquet"]
# df = spark.read_parquet("ubs-datasets",
#                     list_s3,
#                     read_all=False) \
#                     .select(*parquet_cols)

Reading from: ['s3a://ubs-datasets/FRACTAL/data/train/TRAIN-0436_6399-002955400.parquet']


In [36]:
df.printSchema()
print(f"Number of rows: {df.count()}")

root
 |-- xyz: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- Intensity: integer (nullable = true)
 |-- Classification: short (nullable = true)
 |-- Red: integer (nullable = true)
 |-- Green: integer (nullable = true)
 |-- Blue: integer (nullable = true)
 |-- Infrared: integer (nullable = true)
 |-- ReturnNumber: short (nullable = true)
 |-- NumberOfReturns: short (nullable = true)

Number of rows: 90090


Number of rows: 90090


## Preprocessing & Feature Engineering

In [37]:
df.columns

['xyz',
 'Intensity',
 'Classification',
 'Red',
 'Green',
 'Blue',
 'Infrared',
 'ReturnNumber',
 'NumberOfReturns']

In [38]:
preprocessing = PreProcessing(df)
df = preprocessing.split_xyz()

# ## feature engineering
engfeature = FeatureEngineering(df)
df = engfeature.apply_all()

In [39]:
df.columns

['Intensity',
 'Classification',
 'Red',
 'Green',
 'Blue',
 'Infrared',
 'ReturnNumber',
 'NumberOfReturns',
 'x',
 'y',
 'z',
 'height_above_ground',
 'local_density',
 'local_z_std',
 'local_z_range',
 'roughness',
 'return_ratio',
 'is_single_return',
 'is_last_return',
 'ndvi',
 'green_red_ratio',
 'ndwi']

In [40]:
from pyspark.sql.functions import col, sum as _sum
feature_cols = df.columns
null_counts = df.select([
    _sum(col(c).isNull().cast("int")).alias(c) 
    for c in feature_cols
])
null_counts.show()

+---------+--------------+---+-----+----+--------+------------+---------------+---+---+---+-------------------+-------------+-----------+-------------+---------+------------+----------------+--------------+----+---------------+----+
|Intensity|Classification|Red|Green|Blue|Infrared|ReturnNumber|NumberOfReturns|  x|  y|  z|height_above_ground|local_density|local_z_std|local_z_range|roughness|return_ratio|is_single_return|is_last_return|ndvi|green_red_ratio|ndwi|
+---------+--------------+---+-----+----+--------+------------+---------------+---+---+---+-------------------+-------------+-----------+-------------+---------+------------+----------------+--------------+----+---------------+----+
|        0|             0|  0|    0|   0|       0|           0|              0|  0|  0|  0|                  0|            0|          0|            0|        0|           0|               0|             0|   0|              0|   0|
+---------+--------------+---+-----+----+--------+------------+-----

### Load | Models
Prepare the variable for the models

In [41]:
## Import taskmetrics to see how the model is performing
stagemetrics = StageMetrics(spark.spark)
stagemetrics.begin() 

In [42]:
feature_cols = df.drop("Classification").columns  
assembler = VectorAssembler(inputCols=feature_cols,
                            outputCol="features",
                           # handleInvalid="skip" 
                           ) 

scaler = StandardScaler(inputCol="features", outputCol="scaled_features")


In [43]:
# 3. Define model
rf = RandomForestClassifier(featuresCol="scaled_features", 
                            labelCol="Classification",
                            bootstrap=True, 
                            numTrees=60,
                            maxDepth=10)

### Pipeline

In [44]:
pipeline = Pipeline(stages=[assembler, scaler, rf])

### Train 

In [45]:
model = pipeline.fit(df)

In [46]:
model.transform(df).show(2)

+---------+--------------+-----+-----+-----+--------+------------+---------------+----------+-----------+------+--------------------+-------------+-------------------+-----------------+-------------------+------------+----------------+--------------+-------------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+----------+
|Intensity|Classification|  Red|Green| Blue|Infrared|ReturnNumber|NumberOfReturns|         x|          y|     z| height_above_ground|local_density|        local_z_std|    local_z_range|          roughness|return_ratio|is_single_return|is_last_return|               ndvi|   green_red_ratio|              ndwi|            features|     scaled_features|       rawPrediction|         probability|prediction|
+---------+--------------+-----+-----+-----+--------+------------+---------------+----------+-----------+------+--------------------+-------------+-------------------+-----------------+-------

## Hyperparameter tunning

In [19]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

## Inference

In [47]:
df_test = spark.read_parquet("ubs-datasets",
                    "FRACTAL/data/train/TRAIN-1200_6136-008972557.parquet",
                    read_all=False) \
                    .select(*parquet_cols)

preprocessing = PreProcessing(df_test)
df_test = preprocessing.split_xyz()
eng_feature = FeatureEngineering(df_test)
df_test = eng_feature.apply_all()

Reading from: ['s3a://ubs-datasets/FRACTAL/data/train/TRAIN-1200_6136-008972557.parquet']


In [48]:
predictions = model.transform(df_test)
model.transform(df_test).show(2)

+--------------+----------+--------------------+
|Classification|Prediction|         Probability|
+--------------+----------+--------------------+
|             4|       5.0|[0.0,5.6102743770...|
|             4|       5.0|[0.0,5.4542493826...|
+--------------+----------+--------------------+
only showing top 2 rows


In [49]:
model.transform(df_test).show(2)

+---------+--------------+-----+-----+-----+--------+------------+---------------+----------+----------+------------------+-------------------+-------------+-----------------+------------------+-------------------+------------+----------------+--------------+-------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|Intensity|Classification|  Red|Green| Blue|Infrared|ReturnNumber|NumberOfReturns|         x|         y|                 z|height_above_ground|local_density|      local_z_std|     local_z_range|          roughness|return_ratio|is_single_return|is_last_return|               ndvi|   green_red_ratio|                ndwi|            features|     scaled_features|       rawPrediction|         probability|prediction|
+---------+--------------+-----+-----+-----+--------+------------+---------------+----------+----------+------------------+-------------------+-------------+-------------

## Evaluation

In [50]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [51]:
evaluator = MulticlassClassificationEvaluator(
    labelCol = 'Classification',
    predictionCol = 'Prediction',
    metricName = 'accuracy'
)

accuracy = evaluator.evaluate(predictions)
print(f"Test Accuracy = {accuracy:.2f}")

Test Accuracy: 0.518


In [52]:
stagemetrics.end()
# stagemetrics.print_memory_report()
# stagemetrics.print_report()

In [53]:
print(type(str(stagemetrics.print_memory_report())))


Additional stage-level executor metrics (memory usage info updated at each heartbeat):

Stage 14 JVMHeapMemory maxVal bytes => 446825352 (426.1 MB)
Stage 14 OnHeapExecutionMemory maxVal bytes => 0 (0 Bytes)
Stage 16 JVMHeapMemory maxVal bytes => 446825352 (426.1 MB)
Stage 16 OnHeapExecutionMemory maxVal bytes => 0 (0 Bytes)
Stage 19 JVMHeapMemory maxVal bytes => 446825352 (426.1 MB)
Stage 19 OnHeapExecutionMemory maxVal bytes => 0 (0 Bytes)
Stage 23 JVMHeapMemory maxVal bytes => 446825352 (426.1 MB)
Stage 23 OnHeapExecutionMemory maxVal bytes => 0 (0 Bytes)
Stage 24 JVMHeapMemory maxVal bytes => 446825352 (426.1 MB)
Stage 24 OnHeapExecutionMemory maxVal bytes => 0 (0 Bytes)
Stage 26 JVMHeapMemory maxVal bytes => 446825352 (426.1 MB)
Stage 26 OnHeapExecutionMemory maxVal bytes => 0 (0 Bytes)
Stage 27 JVMHeapMemory maxVal bytes => 446825352 (426.1 MB)
Stage 27 OnHeapExecutionMemory maxVal bytes => 0 (0 Bytes)
Stage 29 JVMHeapMemory maxVal bytes => 446825352 (426.1 MB)
Stage 29 OnHeapExe

In [54]:
spark.close()


Spark session stopped.


In [56]:
round(accuracy, 2)

0.52